In [1]:
# All Baselines for Node-Level classification - for left and right temporal lobe nodes
import os
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
from scipy.io import loadmat
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import SVC as SVM

from sklearn.metrics import balanced_accuracy_score
from matplotlib import pyplot as plt

import xgboost
from xgboost import XGBClassifier as xgb

import warnings
warnings.filterwarnings("ignore")


In [2]:
def calculate_metrics(y_pred, y_true):

    return balanced_accuracy_score(y_true, y_pred)

In [3]:
# Root Folder
# root='/home/neil/Lab_work/Jeong_Lab_Multi_Modal_MRI/Left_Hemis/Part_2/'
root='/home/neil/Lab_work/Jeong_Lab_Multi_Modal_MRI/Right_Hemis/Part_2/'

In [6]:
# # Right Hemis Nodes
# def get_list_of_node_nums():
#     node_numbers_with_smote = [
#     "504", "506", "508", "509", "510", "511", "512", "513", "514", "515", "516", "517", "518", "519", "520", "521", "522", "524", "525", "526", "529", "530", "534", "535", "536", "537", "538", "539", "540", "541", "542", "543", "546", "547", "548", "549", "551", "552", "553", "554", "555", "556", "557", "558", "559", "560", "561", "562", "563", "564", "565", "566", "567", "568", "569", "570", "571", "572", "573", "574", "575", "576", "581", "582", "584", "585", "586", "587", "588", "589", "590", "591", "592", "593", "594", "595", "596", "598", "599", "600", "601", "602", "603", "604", "605", "606", "607", "608", "609", "610", "612", "613", "614", "615", "616", "617", "618", "619", "620", "621", "622", "623", "624", "625", "627", "628", "629", "630", "632", "633", "634", "635", "636", "637", "638", "639", "640", "641", "642", "643", "644", "645", "646", "647", "648", "649", "650", "651", "652", "655", "656", "657", "658", "659", "660", "661", "662", "663", "664", "665", "666", "668", "669", "670", "671", "672", "673", "674", "675", "676", "677", "678", "681", "683", "685", "686", "690", "691", "692", "693", "694", "695", "696", "697", "698", "699", "700", "701", "702", "703", "704", "705", "706", "707", "708", "709", "710", "711", "712", "713", "714", "715", "716", "717", "718", "719", "720", "721", "722", "723", "724", "725", "726", "727", "728", "730", "731", "732", "733", "735", "736", "737", "738", "739", "740", "741", "742", "743", "744", "745", "746", "747", "748", "749", "750", "751", "756", "757", "758", "759", "760", "761", "762", "763", "764", "765", "766", "767", "770", "771", "776", "777", "778", "779", "780", "781", "782", "783", "784", "785", "786", "787", "788", "789", "790", "791", "792", "793", "795", "796", "797", "798", "799", "800", "801", "802", "803", "804", "805", "806", "808", "809", "810", "811", "812", "813", "816", "817", "818", "819", "820", "821", "822", "823", "824", "825", "826", "827", "828", "829", "830", "831", "832", "834", "835", "836", "837", "838", "839", "841", "842", "843", "844", "845", "846", "847", "848", "849", "850", "851", "852", "853", "854", "855", "856", "857", "858", "859", "860", "861", "862", "863", "864", "865", "866", "867", "868", "869", "870", "871", "872", "873", "874", "875", "877", "878", "879", "880", "881", "882", "883", "885", "886", "887", "888", "889", "890", "891", "892", "893", "894", "895", "896", "897", "898", "899", "900", "901", "902", "903", "904", "905", "906", "907", "908", "909", "910", "911", "912", "913", "914", "915", "916", "917", "918", "919", "920", "921", "922", "923", "924", "925", "926", "927", "928", "929", "930", "931", "932", "933", "934", "935", "937", "938", "939", "940", "941", "942", "943", "944", "945", "946", "947", "948", "949", "950", "951", "952", "953", "954", "955", "956", "957", "958", "960", "961", "962", "963", "964", "965", "968", "969", "970", "971", "973", "974", "975", "976", "977", "978", "979", "980", "981", "982", "983"
#     ]

#     return node_numbers_with_smote

# # Left Hemis Nodes
# def get_list_of_node_nums():
#     node_numbers_with_smote = [
#     "6", "11", "12", "14", "18", "19", "20", "33", "34", "35", "36", "39", "41", "42", "43", "44", "45", "46", "47", "48", "49", "51", "52", "53", "54", "55", "56", "57", "58", "59", "60", "61", "62", "63", "64", "66", "68", "79", "80", "81", "84", "85", "86", "87", "88", "90", "91", "93", "94", "95", "96", "97", "98", "101", "102", "103", "104", "108", "109", "111", "120", "121", "122", "123", "124", "125", "126", "127", "128", "129", "130", "131", "140", "144", "145", "147", "148", "150", "151", "155", "156", "158", "159", "160", "163", "164", "165", "166", "169", "175", "176", "177", "192", "193", "194", "195", "197", "198", "199", "200", "202", "204", "205", "211", "213", "214", "216", "217", "220", "221", "222", "224", "225", "226", "227", "228", "229", "230", "231", "232", "233", "234", "235", "238", "239", "240", "241", "245", "246", "247", "248", "251", "252", "253", "256", "257", "260", "261", "275", "290", "291", "292", "294", "295", "296", "297", "298", "299", "301", "302", "303", "304", "305", "306", "316", "320", "321", "322", "326", "331", "332", "334", "335", "336", "337", "338", "339", "340", "343", "346", "349", "352", "353", "354", "355", "356", "357", "359", "360", "361", "362", "363", "364", "365", "366", "367", "368", "369", "370", "371", "372", "374", "375", "376", "377", "378", "381", "382", "383", "384", "386", "387", "388", "389", "390", "391", "394", "395", "396", "397", "398", "399", "400", "401", "402", "403", "404", "405", "406", "408", "409", "410", "411", "412", "413", "414", "415", "416", "417", "418", "419", "420", "421", "422", "423", "424", "426", "427", "428", "429", "430", "431", "432", "433", "435", "436", "437", "438", "439", "440", "441", "442", "443", "444", "445", "446", "447", "448", "450", "451", "452", "453", "454", "455", "456", "458", "459", "460", "461", "462", "463", "464", "465", "466", "467", "469", "470", "471", "472", "473", "474", "475", "476", "477", "478", "479"
#     ]

#     return node_numbers_with_smote

In [4]:
# Left Hemis Nodes
def get_list_of_node_nums():
    node_numbers_with_smote = [
     "504", "506", "508", "509", "510", "511", "512", "513", "514", "515", "516", "517", "518", "519", "520", "521", "522", "524", "525", "526", "529", "530", "534", "535", "536", "537", "538", "539", "540", "541", "542", "543", "546", "547", "548", "549", "551", "552", "553", "554", "555", "556", "557", "558", "559", "560", "561", "562", "563", "564", "565", "566", "567", "568", "569", "570", "571", "572", "573", "574", "575", "576", "581", "582", "584", "585", "586", "587", "588", "589", "590", "591", "592", "593", "594", "595", "596", "598", "599", "600", "601", "602", "603", "604", "605", "606", "607", "608", "609", "610", "612", "613", "614", "615", "616", "617", "618", "619", "620", "621", "622", "623", "624", "625", "627", "628", "629", "630", "632", "633", "634", "635", "636", "637", "638", "639", "640", "641", "642", "643", "644", "645", "646", "647", "648", "649", "650", "651", "652", "655", "656", "657", "658", "659", "660", "661", "662", "663", "664", "665", "666", "668", "669", "670", "671", "672", "673", "674", "675", "676", "677", "678", "681", "683", "685", "686", "690", "691", "692", "693", "694", "695", "696", "697", "698", "699", "700", "701", "702", "703", "704", "705", "706", "707", "708", "709", "710", "711", "712", "713", "714", "715", "716", "717", "718", "719", "720", "721", "722", "723", "724", "725", "726", "727", "728", "730", "731", "732", "733", "735", "736", "737", "738", "739", "740", "741", "742", "743", "744", "745", "746", "747", "748", "749", "750", "751", "756", "757", "758", "759", "760", "761", "762", "763", "764", "765", "766", "767", "770", "771", "776", "777", "778", "779", "780", "781", "782", "783", "784", "785", "786", "787", "788", "789", "790", "791", "792", "793", "795", "796", "797", "798", "799", "800", "801", "802", "803", "804", "805", "806", "808", "809", "810", "811", "812", "813", "816", "817", "818", "819", "820", "821", "822", "823", "824", "825", "826", "827", "828", "829", "830", "831", "832", "834", "835", "836", "837", "838", "839", "841", "842", "843", "844", "845", "846", "847", "848", "849", "850", "851", "852", "853", "854", "855", "856", "857", "858", "859", "860", "861", "862", "863", "864", "865", "866", "867", "868", "869", "870", "871", "872", "873", "874", "875", "877", "878", "879", "880", "881", "882", "883", "885", "886", "887", "888", "889", "890", "891", "892", "893", "894", "895", "896", "897", "898", "899", "900", "901", "902", "903", "904", "905", "906", "907", "908", "909", "910", "911", "912", "913", "914", "915", "916", "917", "918", "919", "920", "921", "922", "923", "924", "925", "926", "927", "928", "929", "930", "931", "932", "933", "934", "935", "937", "938", "939", "940", "941", "942", "943", "944", "945", "946", "947", "948", "949", "950", "951", "952", "953", "954", "955", "956", "957", "958", "960", "961", "962", "963", "964", "965", "968", "969", "970", "971", "973", "974", "975", "976", "977", "978", "979", "980", "981", "982", "983"
    ]

    return node_numbers_with_smote

In [5]:
def load_train_data(root: str, node_num: str):

    train_path = os.path.join(root, 'Node_'+node_num, 'Aug_Train_Data', 'ALL_Patients')  
    x_file = f"X_train_aug"
    y_file = f"Y_train_aug"
    x_mat_name = "X_aug_train"
    y_mat_name = "Y_aug_train"  

    raw_path_x = os.path.join(train_path, f"{x_file}.mat")
    raw_path_y = os.path.join(train_path, f"{y_file}.mat")

    # Load the data from .mat files
    X_mat_l = loadmat(raw_path_x)
    X_mat = X_mat_l[x_mat_name]

    Y_mat_l = loadmat(raw_path_y)
    Y_mat = Y_mat_l[y_mat_name]
    Y_mat = Y_mat.reshape(Y_mat.shape[1],)

    return X_mat, Y_mat

In [6]:
def load_test_data(root: str, node_num: str):

    val_path = os.path.join(root, 'Node_'+node_num, 'Orig_Val_Data', 'ALL_Patients')  
    x_file = f"X_valid_orig"
    y_file = f"Y_valid_orig"
    x_mat_name = "X_orig_valid"
    y_mat_name = "Y_orig_valid"  

    raw_path_x = os.path.join(val_path, f"{x_file}.mat")
    raw_path_y = os.path.join(val_path, f"{y_file}.mat")

    # Load the data from .mat files
    X_mat_l = loadmat(raw_path_x)
    X_mat = X_mat_l[x_mat_name]

    Y_mat_l = loadmat(raw_path_y)
    Y_mat = Y_mat_l[y_mat_name]
    Y_mat = Y_mat.reshape(Y_mat.shape[1],)

    return X_mat, Y_mat

In [7]:
# choose model
# model = 'MLP'
# model = 'RF'
model = 'XGB'
# model = 'SVM'

In [8]:
# Main loop to run the baseline models over all the nodes (for all 5 trials)

node_numbers_with_smote = get_list_of_node_nums()

for node_num in node_numbers_with_smote:
    
    print(f'Node num: {node_num}')

    num_trials = 5
    
    # Create empty lists to store results for bal_accuracy
    val_bal_acc_list = [[] for _ in range(num_trials)]

    # load the data for the given node
    X_train, Y_train = load_train_data(root, node_num)
    
    X_test, Y_test = load_test_data(root, node_num)

    # Define the model
    if model == 'MLP':
        clf = MLP(hidden_layer_sizes=(256,), learning_rate_init=0.01, random_state=None, max_iter=1000, early_stopping=False)
    elif model == 'RF':
        clf = RF(n_estimators=100, random_state=None,)
    elif model == 'XGB':
        clf = xgb(objective='binary:logistic',max_depth = 5,n_estimators = 10, random_state=None) # This is used
        # clf = xgb(objective='binary:logistic', n_estimators = 100, random_state=None) # with _1.xlsx extension (NOT Used)
    elif model == 'SVM':
        clf = SVM(C=1.0, kernel='rbf', max_iter=-1, random_state=None)
    else:
        raise NotImplementedError("Unknown Model.")

    # Run 5 trials for each node
    for i in range(num_trials):
        print(f'Training Trial {i+1} of Node number {node_num}')

        # Train the model
        clf.fit(X_train, Y_train)

        # Test the model
        print(f'Evaluating Trial {i+1} of Node number: {node_num}')
        y_true = Y_test
        y_pred = clf.predict(X_test)

        # Evaluate Trained Model with evaluation metrics
        bal_acc = calculate_metrics(y_pred, y_true)  
        print(f"Balanced Accuracy: {bal_acc}")

        val_bal_acc_list[i].append(bal_acc) 

    # Save the results in a dataframe
    # Combine data
    row_data_val = [node_num] + [val_bal_acc_list[j][0] for j in range(num_trials)]

    # Create a DataFrame
    headers_val = ['Node #', 'Val_Bal_Acc_1', 'Val_Bal_Acc_2', 'Val_Bal_Acc_3', 'Val_Bal_Acc_4', 'Val_Bal_Acc_5']

    df_val = pd.DataFrame([row_data_val], columns=headers_val)

    # Saving to Excel
    path = "/home/neil/Lab_work/Jeong_Lab_Multi_Modal_MRI/Right_Hemis/Part_2/"
    # path = "/home/neil/Lab_work/Jeong_Lab_Multi_Modal_MRI/Left_Hemis/Part_2/"  
    save_path = os.path.join(path, "Node_"+str(node_num), "Baseline_Results")

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    filename_val = model + "_results_val.xlsx"
    save_filepath_val = os.path.join(save_path, filename_val)

    df_val.to_excel(save_filepath_val, index=False, sheet_name='Sheet1')

    print("\nDone!")



Node num: 504
Training Trial 1 of Node number 504
Evaluating Trial 1 of Node number: 504
Balanced Accuracy: 1.0
Training Trial 2 of Node number 504
Evaluating Trial 2 of Node number: 504
Balanced Accuracy: 1.0
Training Trial 3 of Node number 504
Evaluating Trial 3 of Node number: 504
Balanced Accuracy: 1.0
Training Trial 4 of Node number 504
Evaluating Trial 4 of Node number: 504
Balanced Accuracy: 1.0
Training Trial 5 of Node number 504
Evaluating Trial 5 of Node number: 504
Balanced Accuracy: 1.0

Done!
Node num: 506
Training Trial 1 of Node number 506
Evaluating Trial 1 of Node number: 506
Balanced Accuracy: 0.9
Training Trial 2 of Node number 506
Evaluating Trial 2 of Node number: 506
Balanced Accuracy: 0.9
Training Trial 3 of Node number 506
Evaluating Trial 3 of Node number: 506
Balanced Accuracy: 0.9
Training Trial 4 of Node number 506
Evaluating Trial 4 of Node number: 506
Balanced Accuracy: 0.9
Training Trial 5 of Node number 506
Evaluating Trial 5 of Node number: 506
Balanced

In [9]:
# Combine all node results into one dataframe

# Define the paths of your Excel files
base_path = "/home/neil/Lab_work/Jeong_Lab_Multi_Modal_MRI/Right_Hemis/Part_2/"
# base_path = "/home/neil/Lab_work/Jeong_Lab_Multi_Modal_MRI/Left_Hemis/Part_2/"  

# For FULL modality Only
node_nums = get_list_of_node_nums()

file_paths_val = []

for node_num in node_nums:
    file_path_val = os.path.join(base_path, "Node_"+node_num, "Baseline_Results", model + "_results_val.xlsx") # For FULL modality Only
    file_paths_val.append(file_path_val)

# Initialize an empty DataFrame
combined_df_val = pd.DataFrame()

# Loop through the files and stack the rows
for path in file_paths_val:
    # Load the Excel file
    df = pd.read_excel(path)  

    # Stack the rows
    combined_df_val = pd.concat([combined_df_val, df], axis=0) # For FULL modality Only

# Reset the index to avoid duplicate row indices
combined_df_val = combined_df_val.reset_index(drop=True)

# Save the combined DataFrame to a new Excel file
combined_df_val.to_excel(model+'_RightHemis_val_FULL_modality_Only.xlsx', index=False)
# combined_df_val.to_excel(model+'_LeftHemis_val_FULL_modality_Only.xlsx', index=False)